Book Einführung in Optimirungsmodelle from Nathan Sudermann-Merx page 120

In [1]:
import polars as pl
import pyoframe as pf

In [2]:
df = pl.read_csv('./elspot-prices_2021_hourly_eur.csv', try_parse_dates=True)
# use time zone to fix DST problem in the end of October where one hour is "duplicated"
time_tick = pl.datetime_range(start=df.select("Date").min().select(Date=pl.col('Date') - pl.duration(hours=3)),
                              end=df.select("Date").max().select(Date=pl.col('Date') + pl.duration(days=1)),
                              interval='1h', eager=True, time_zone='UTC', closed='none').dt.convert_time_zone(
    'Europe/Prague').alias('tick').to_frame().filter(
    pl.col("tick").is_between(pl.datetime(2021, 1, 1, time_zone='Europe/Prague'),
                              pl.datetime(2022, 1, 1, time_zone='Europe/Prague'), closed='left'))

In [3]:
# fix DST problem at the end of March where one hour is "missing" so it is filled with null
# kids, please do not do use dates without time zone at home
df = df.drop_nulls()


In [4]:
hourly_prices = pl.concat((time_tick, df), how='horizontal').select(pl.col('tick'),
                                                                    pl.col('DE-LU').str.replace(',', '.',
                                                                                                literal=True).str.to_decimal().alias(
                                                                        'price'))

In [5]:
pump_max = 70
turb_max = 90
effic = 0.75
storage_capacity = 630
storage_lower_bound = 100
storage_level_init = 300
storage_level_final = 300
tick_with_initial = pl.concat((time_tick.min().with_columns(pl.col("tick").dt.offset_by('-1h')), time_tick))

Decision variables:

In [6]:
m = pf.Model("unit commitment problem", solver="highs", use_var_names=True)

m.Pump = pf.Variable(time_tick[['tick']], vtype=pf.VType.BINARY)
# ub is redundant since it will be set also in logical condition that pump and turbine cannot work at the same time
m.Turb = pf.Variable(time_tick[['tick']], lb=0, ub=turb_max)
m.Storage_level = pf.Variable(tick_with_initial, lb=storage_lower_bound, ub=storage_capacity)
m.initial_storage_level = m.Storage_level.filter(
    pl.col('tick') == tick_with_initial[['tick']].min()) == storage_level_init
m.final_storage_level = m.Storage_level.filter(pl.col('tick') == time_tick[['tick']].max()) == storage_level_final

Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms


In [7]:

previous_hour_storage_level = m.Storage_level.with_columns(pl.col('tick').dt.offset_by('1h'))


In [8]:

m.intermediate_storage_level = m.Storage_level.drop_unmatched() == (previous_hour_storage_level.drop_unmatched() + (
            m.Pump * pump_max * effic - m.Turb).drop_unmatched()).drop_unmatched()
m.pump_and_turbine_xor = m.Turb <= (1 - m.Pump) * turb_max

m.maximize = pf.sum((m.Turb - pump_max * m.Pump) * hourly_prices)

m.optimize()
m.write('unit_commitment_problem.lp')
m.Turb.solution.write_csv('turbine.csv')
m.Storage_level.solution.write_csv('storage.csv')
m.Pump.solution.write_csv('pump.csv')

Hessian has dimension 26282 but no nonzeros, so is ignored
MIP  has 17522 rows; 26282 cols; 61324 nonzeros; 8760 integer variables (8760 binary)
Coefficient ranges:
  Matrix [1e+00, 3e+02]
  Cost   [1e-02, 4e+04]
  Bound  [1e+00, 6e+02]
  RHS    [0e+00, 0e+00]
Presolving model
17520 rows, 26279 cols, 52558 nonzeros  0s
17518 rows, 26277 cols, 52554 nonzeros  0s

Solving MIP model with:
   17518 rows
   26277 cols (8760 binary, 0 integer, 0 implied int., 17517 continuous)
   52554 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   